In [10]:
import numpy as np
import pandas as pd
from plotnine import *
from datetime import datetime

In [30]:
def sekundeid_südaööst(aeg: str):
    aeg = datetime.fromisoformat(aeg)
    return (aeg - aeg.replace(hour=0, minute=0)).total_seconds()

In [48]:
df = pd.read_csv('spioonid.csv')
df['päeva_sekund'] = df['kuupäev'].apply(sekundeid_südaööst)
df['nädal'] = df['kuupäev'].apply(lambda x: datetime.isocalendar(datetime.fromisoformat(x)).week)
df['söögigraafik_nädal'] = df['nädal'].apply(lambda x: x % 7)

df

,kuupäev,päeva_sekund,nädal,söögigraafik_nädal
0,2023-07-23T14:11:00,51060.0,29,1
1,2023-07-23T17:30:00,63000.0,29,1
2,2023-07-23T18:32:00,66720.0,29,1
3,2023-07-23T21:34:00,77640.0,29,1
4,2023-07-23T22:05:00,79500.0,29,1
5,2023-07-24T19:45:00,71100.0,30,2
6,2023-07-24T19:54:00,71640.0,30,2
7,2023-07-26T19:07:00,68820.0,30,2
8,2023-07-27T22:07:00,79620.0,30,2
9,2023-07-27T22:13:00,79980.0,30,2
